# Merge tunings for r3 Release

Here we will merge all models from v8 (Zee) and v1 (jpsi) tunings.

All tunings were trained using run2 schemma.

In [17]:
from kolmov import crossval_table, get_color_fader, fit_table
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from ROOT import TEnv
from pprint import pprint
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
v8_path = "/Volumes/castor/tuning_data/Zee/production/TrigL2_20180125_v8"
v1_path = "/Volumes/castor/tuning_data/Jpsi/production/TrigL2_20210227_v1"

In [19]:
def merge( path1, path2 , output, max_pileup = [100,100]):

    env1 = TEnv(path1)
    env2 = TEnv(path2)
    def treat_float( env, key ):
        return [float(value) for value in  env.GetValue(key, '').split('; ')]

    def treat_string( env, key ):
        return [str(value) for value in  env.GetValue(key, '').split('; ')]
    
    # merge models
    model_etmin_vec= treat_float( env1,'Model__etmin' )+treat_float( env2,'Model__etmin' )
    model_etmax_vec= treat_float( env1,'Model__etmax' )+treat_float( env2,'Model__etmax' )
    model_etamin_vec= treat_float( env1,'Model__etamin' )+treat_float( env2,'Model__etamin' )
    model_etamax_vec= treat_float( env1,'Model__etamax' )+treat_float( env2,'Model__etamax' )
    model_paths =  treat_string( env1, 'Model__path' )+ treat_string( env2, 'Model__path' )
    slopes = treat_float( env1, 'Threshold__slope' )+treat_float( env2, 'Threshold__slope' )
    offsets = treat_float( env1, 'Threshold__offset' )+treat_float( env2, 'Threshold__slope' )
    
    file = TEnv(path1)
    def list_to_str( l ):
        s = str()
        for ll in l:
            s+=str(ll)+'; '
        return s[:-2]

    max_pileup1 = [max_pileup[0]]*len(treat_string( env1, 'Model__path' ))
    max_pileup2 = [max_pileup[1]]*len(treat_string( env2, 'Model__path' ))
    

    
    # Write the config file
    file = TEnv( 'ringer' )
    file.SetValue( "__name__", 'should_be_filled' )
    file.SetValue( "__version__", 'should_be_filled' )
    file.SetValue( "__operation__", 'should_be_filled' )
    file.SetValue( "__signature__", 'should_be_filled' )
    file.SetValue( "Model__size"  , str(len(model_paths)) )
    file.SetValue( "Model__etmin" , list_to_str(model_etmin_vec) )
    file.SetValue( "Model__etmax" , list_to_str(model_etmax_vec) )
    file.SetValue( "Model__etamin", list_to_str(model_etamin_vec) )
    file.SetValue( "Model__etamax", list_to_str(model_etamax_vec) )
    file.SetValue( "Model__path"  , list_to_str( model_paths ) )
    file.SetValue( "Threshold__size"  , str(len(model_paths)) )
    file.SetValue( "Threshold__etmin" , list_to_str(model_etmin_vec) )
    file.SetValue( "Threshold__etmax" , list_to_str(model_etmax_vec) )
    file.SetValue( "Threshold__etamin", list_to_str(model_etamin_vec) )
    file.SetValue( "Threshold__etamax", list_to_str(model_etamax_vec) )
    file.SetValue( "Threshold__slope" , list_to_str(slopes) )
    file.SetValue( "Threshold__offset", list_to_str(offsets) )
    file.SetValue( "Threshold__MaxAverageMu", list_to_str(max_pileup1+max_pileup2))
    file.WriteFile(output)




In [20]:
merge(v1_path+'/ElectronRingerVeryLooseTriggerConfig.conf',
      v8_path+'/ElectronRingerVeryLooseTriggerConfig.conf',
      'ElectronRingerVeryLooseTriggerConfig.conf',
      [50, 100])

In [21]:
merge(v1_path+'/ElectronRingerLooseTriggerConfig.conf',
      v8_path+'/ElectronRingerLooseTriggerConfig.conf',
      'ElectronRingerLooseTriggerConfig.conf',
      [50, 100])

In [22]:
merge(v1_path+'/ElectronRingerMediumTriggerConfig.conf',
      v8_path+'/ElectronRingerMediumTriggerConfig.conf',
      'ElectronRingerMediumTriggerConfig.conf',
      [50, 100])

In [23]:
merge(v1_path+'/ElectronRingerTightTriggerConfig.conf',
      v8_path+'/ElectronRingerTightTriggerConfig.conf',
      'ElectronRingerTightTriggerConfig.conf',
      [50, 100])

In [24]:
!mkdir TrigL2_20210227_r3
!cp -r /Volumes/castor/tuning_data/Zee/production/TrigL2_20180125_v8/models TrigL2_20210227_r3
!cp -r /Volumes/castor/tuning_data/Jpsi/production/TrigL2_20210227_v1/models/* TrigL2_20210227_r3/models
!mv *.conf TrigL2_20210227_r3